In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, datasets

In [ ]:
if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')
print('Using PyTorch version:', torch.__version__, ' Device:', DEVICE)

Using PyTorch version: 1.8.1+cu101  Device: cuda


In [ ]:
BATCH_SIZE = 32
EPOCHS = 10

In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ]),
    'val': transforms.Compose([
        transforms.CenterCrop(224),
        transforms.Resize(256),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ]),
}

image_datasets = {x: datasets.ImageFolder("/content/drive/MyDrive/data/hymenoptera_data", data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size = BATCH_SIZE, num_workers = 0, shuffle = True) for x in ['train', 'val']}

In [ ]:
def train(model, train_loader, optimizer, log_interval):
    model.train()
    for batch_idx, (image, label) in enumerate(train_loader):
        image = image.to(DEVICE)
        label = label.to(DEVICE)
        optimizer.zero_grad()
        output = model(image)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()

        if batch_idx % log_interval == 0:
            print("Train Epoch: {} [{}/{} ({:.0f}%)]\tTrain Loss: {:.6f}".format(
                epoch, batch_idx * len(image), 
                len(train_loader.dataset), 100. * batch_idx / len(train_loader), 
                loss.item()))

In [ ]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for image, label in test_loader:
            image = image.to(DEVICE)
            label = label.to(DEVICE)
            output = model(image)
            test_loss += criterion(output, label).item()
            prediction = output.max(1, keepdim = True)[1]
            correct += prediction.eq(label.view_as(prediction)).sum().item()
    
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [ ]:
import torchvision.models as models
model = models.resnet18(pretrained = False).cuda()
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)
model = model.cuda()

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)
criterion = nn.CrossEntropyLoss()

print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
for epoch in range(1, EPOCHS + 1):
    train(model, dataloaders["train"], optimizer, log_interval = 5)
    test_loss, test_accuracy = evaluate(model, dataloaders["val"])
    print("\n[EPOCH: {}], \tTest Loss: {:.4f}, \tTest Accuracy: {:.2f} % \n".format(
        epoch, test_loss, test_accuracy))

Train Epoch: 1 [0/397 (0%)]	Train Loss: 0.661970
Train Epoch: 1 [160/397 (38%)]	Train Loss: 0.832303
Train Epoch: 1 [320/397 (77%)]	Train Loss: 0.720373

[EPOCH: 1], 	Test Loss: 0.0220, 	Test Accuracy: 60.20 % 

Train Epoch: 2 [0/397 (0%)]	Train Loss: 0.612502
Train Epoch: 2 [160/397 (38%)]	Train Loss: 0.643379
Train Epoch: 2 [320/397 (77%)]	Train Loss: 0.767263

[EPOCH: 2], 	Test Loss: 0.0218, 	Test Accuracy: 59.45 % 

Train Epoch: 3 [0/397 (0%)]	Train Loss: 0.625953
Train Epoch: 3 [160/397 (38%)]	Train Loss: 0.482991
Train Epoch: 3 [320/397 (77%)]	Train Loss: 0.633768

[EPOCH: 3], 	Test Loss: 0.0216, 	Test Accuracy: 63.73 % 

Train Epoch: 4 [0/397 (0%)]	Train Loss: 0.624910
Train Epoch: 4 [160/397 (38%)]	Train Loss: 0.602476
Train Epoch: 4 [320/397 (77%)]	Train Loss: 0.703433

[EPOCH: 4], 	Test Loss: 0.0216, 	Test Accuracy: 59.95 % 

Train Epoch: 5 [0/397 (0%)]	Train Loss: 0.819259
Train Epoch: 5 [160/397 (38%)]	Train Loss: 0.599229
Train Epoch: 5 [320/397 (77%)]	Train Loss: 0.670239

In [ ]:
model = models.resnet18(pretrained = True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)
model = model.cuda()

optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)
EPOCHS = 10
for epoch in range(1, EPOCHS + 1):
    train(model, dataloaders["train"], optimizer, log_interval = 5)
    valid_loss, valid_accuracy = evaluate(model, dataloaders["val"])
    print("\n[EPOCH: {}], \tTest Loss: {:.4f}, \tTest Accuracy: {:.2f} % \n".format(
        epoch, valid_loss, valid_accuracy))

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth



Train Epoch: 1 [0/397 (0%)]	Train Loss: 0.907907
Train Epoch: 1 [160/397 (38%)]	Train Loss: 0.667259
Train Epoch: 1 [320/397 (77%)]	Train Loss: 0.734559

[EPOCH: 1], 	Test Loss: 0.0209, 	Test Accuracy: 64.99 % 

Train Epoch: 2 [0/397 (0%)]	Train Loss: 0.556805
Train Epoch: 2 [160/397 (38%)]	Train Loss: 0.508806
Train Epoch: 2 [320/397 (77%)]	Train Loss: 0.475531

[EPOCH: 2], 	Test Loss: 0.0158, 	Test Accuracy: 78.34 % 

Train Epoch: 3 [0/397 (0%)]	Train Loss: 0.412044
Train Epoch: 3 [160/397 (38%)]	Train Loss: 0.443350
Train Epoch: 3 [320/397 (77%)]	Train Loss: 0.412454

[EPOCH: 3], 	Test Loss: 0.0130, 	Test Accuracy: 86.15 % 

Train Epoch: 4 [0/397 (0%)]	Train Loss: 0.366188
Train Epoch: 4 [160/397 (38%)]	Train Loss: 0.300205
Train Epoch: 4 [320/397 (77%)]	Train Loss: 0.458015

[EPOCH: 4], 	Test Loss: 0.0114, 	Test Accuracy: 87.41 % 

Train Epoch: 5 [0/397 (0%)]	Train Loss: 0.536917
Train Epoch: 5 [160/397 (38%)]	Train Loss: 0.316361
Train Epoch: 5 [320/397 (77%)]	Train Loss: 0.27454